# Homework 4 : Applied Machine Learning

In [74]:
# imports
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
%matplotlib inline

In [75]:
# import data2
df = pd.read_csv('CrowdstormingDataJuly1st.csv', index_col=0)
df.head()

,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
playerShort,,,,,,,,,,,,,,,,,,,,,
lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [129]:
# drop rows for which we have no photo (and thus no skin rating)
df.dropna(subset=['photoID'], inplace=True)

# group by players
group_by_players = df.groupby(level=0)

# build the data used for later analysis

# data
player_description = group_by_players.agg({'leagueCountry': lambda x: x.value_counts().index[0],
                                           'position': 'first',
                                           'height': 'mean', 
                                           'weight': 'mean',
                                           'games': 'sum',
                                           'victories': 'sum',
                                           'ties': 'sum',
                                           'defeats': 'sum',
                                           'goals': 'sum',
                                           'yellowCards': 'sum',
                                           'yellowReds': 'sum',
                                           'redCards': 'sum',
                                           'rater1': 'first',
                                           'rater2': 'first'
                                          })
player_description = pd.get_dummies(player_description, dummy_na= True, columns = ['leagueCountry', 'position'])
player_description.head()

,goals,redCards,weight,ties,rater1,games,defeats,yellowReds,height,victories,...,position_Center Midfielder,position_Defensive Midfielder,position_Goalkeeper,position_Left Fullback,position_Left Midfielder,position_Left Winger,position_Right Fullback,position_Right Midfielder,position_Right Winger,position_nan
playerShort,,,,,,,,,,,,,,,,,,,,,
aaron-hughes,9,0,71.0,179,0.25,654,228,0,182.0,247,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaron-hunt,62,1,73.0,73,0.00,336,122,0,183.0,141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaron-lennon,31,0,63.0,97,0.25,412,115,0,165.0,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
aaron-ramsey,39,1,76.0,42,0.00,260,68,0,178.0,150,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abdelhamid-el-kaoutari,1,2,73.0,40,0.25,124,43,4,180.0,41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
player_description.isnull().sum()

goals                             0
redCards                          0
weight                           21
ties                              0
rater1                            0
games                             0
defeats                           0
yellowReds                        0
height                            3
victories                         0
rater2                            0
yellowCards                       0
leagueCountry_England             0
leagueCountry_France              0
leagueCountry_Germany             0
leagueCountry_Spain               0
leagueCountry_nan                 0
position_Attacking Midfielder     0
position_Center Back              0
position_Center Forward           0
position_Center Midfielder        0
position_Defensive Midfielder     0
position_Goalkeeper               0
position_Left Fullback            0
position_Left Midfielder          0
position_Left Winger              0
position_Right Fullback           0
position_Right Midfielder   

We can see that we still have missing values in these two fields: height and weight. We have two toptions to deal with this: either get rid of the players for which we don't have those values (in the worst case scenario, that's 24 players less) or we try to interpolate their weight and height by taking the median or the mean for example. This can introduce bias so we chose the first option

In [131]:
player_description.dropna(subset=['weight', 'height'], inplace=True)

# labels
player_rating = player_description['rater1']

player_description.drop(['rater1', 'rater2'], axis=1, inplace=True)


## Training the classifier

In [281]:
X = player_description.as_matrix()
y = (player_rating*4).as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
model = RandomForestClassifier(n_estimators=30, max_depth=10)
c = sklearn.cross_validation.cross_val_score(model, X_train, y_train, cv=10)
print("Cross validation error:", np.mean(c)*100)
model.fit(X_train, y_train)

te_pred = model.predict(X_test)
tr_pred = model.predict(X_train)

tr_error = tr_pred == y_train
tr_error = np.sum(tr_error)/len(tr_error)

te_error = te_pred == y_test
te_error = np.sum(te_error)/len(te_error)

print("Train error:", tr_error*100)
print("Test error:", te_error*100)


Cross validation error: 42.7110535149
Train error: 82.0184790334
Test error: 44.5859872611


In [101]:
set(y)

{0.0, 1.0, 2.0, 3.0, 4.0}